In [1]:
# standard libraries
import numpy as np
import math
from pprint import pprint as pp
from matplotlib.pyplot import *
from matplotlib.colors import LogNorm
%matplotlib inline

In [2]:
# user library
from bayesee.imaging.filter import *
from bayesee.imaging.image import *
from bayesee.operation.nb2d import *
from bayesee.operation.dp import *
from bayesee.operation.mathfunc import *

In [3]:
# stimulus parameters
row = col = 128
ppd = 120
cpd = 4
cpi_v = row * cpd / ppd

In [4]:
# simulation parameters
n_background = 100000
background_mean = 0
background_std = 1

In [5]:
# templates
rc_template = hann_window(row, col, row/2)
rc_template /= nb2dot(rc_template, rc_template)

sine_template = hann_window(row, col, row/2) * sine_wave(row, col, (cpi_v,0))
sine_template /= nb2dot(sine_template, sine_template)

plaid_template = sine_template * hann_window(row, col, row/2) * sine_wave(row, col, (0, cpi_v))
plaid_template /= nb2dot(plaid_template, plaid_template)

In [6]:
# backgrounds
one_f_0_noise = np.zeros((row, col, n_background))
one_f_p5_noise = np.zeros((row, col, n_background))
one_f_noise = np.zeros((row, col, n_background))
one_f_2_noise = np.zeros((row, col, n_background))
one_f_5_noise = np.zeros((row, col, n_background))

for i in range(n_background):
    one_f_0_noise[:,:,i] = power_noise(row, col, 0, background_mean, background_std)
    one_f_p5_noise[:,:,i] = power_noise(row, col, -0.5, background_mean, background_std)
    one_f_noise[:,:,i] = power_noise(row, col, -1, background_mean, background_std)
    one_f_2_noise[:,:,i] = power_noise(row, col, -2, background_mean, background_std)
    one_f_5_noise[:,:,i] = power_noise(row, col, -5, background_mean, background_std)

In [7]:
# window
hann = flat_top_hann_window(row, col, [0.45 * row, 0.5 * row])

In [8]:
# simulation function
def amp_sim_spat_sim(template, images, window):
    n_background = images.shape[2]
    spat_sim = np.zeros((n_background,))
    amp_sim = np.zeros((n_background,))
    
    for i in range(n_background):
        background = images[:,:,i]
        spat_sim[i] = spatial_cosine_similarity(template, background, window)
        amp_sim[i] = amplitude_cosine_similarity(template, background, window)
            
    return spat_sim, amp_sim

In [9]:
# simulation
templates = [rc_template, sine_template, plaid_template]
backgrounds = [one_f_0_noise, one_f_p5_noise, one_f_noise, one_f_2_noise, one_f_5_noise]

spat_sim_all = np.zeros((len(templates), len(backgrounds), n_background))
amp_sim_all = np.zeros_like(spat_sim_all)

for t, template in enumerate(templates):
    for b, background in enumerate(backgrounds):
        spat_sim_all[t,b,:], amp_sim_all[t,b,:] = amp_sim_spat_sim(template, background, hann)
        
np.savez('1-f-noise-spat-sim-amp-sim', spat_sim_all=spat_sim_all, amp_sim_all=amp_sim_all)

In [5]:
# plot function
def plot_spat_sim_amp_sim(spat_sim_all, amp_sim_all, pargs):
    n_templates, n_backgrounds, _ = spat_sim_all.shape
    n_rows, n_cols = pargs['n_rows'], pargs['n_cols']
    fig, axs = subplots(nrows=n_rows, ncols=n_cols, figsize=pargs['figsize'], constrained_layout=True)
    
    for t in range(n_templates):
        x_max = np.abs(spat_sim_all[t,:,:]).max()
        y_max = amp_sim_all[t,:,:].max() * 1.2
        for b in range(n_backgrounds):
    
            if n_templates == 1 or n_backgrounds == 1:
                idxes = b if n_templates == 1 else t
            else:
                idxes = t, b
            
            h2d = axs[idxes].hist2d(spat_sim_all[t,b,:], amp_sim_all[t,b,:], bins=50, cmap=cm.get_cmap('gray').reversed(), alpha=1, range=[[-x_max, x_max], [0,y_max]], edgecolors='none', norm=LogNorm(clip=True))
            axs[idxes].text(0.5,0.9,pargs['legends'][t][b], transform=axs[idxes].transAxes, c='k')
            # axs[idxes].scatter(spat_sim_all[t,b,:], amp_sim_all[t,b,:], s=pargs['markersize'], marker=pargs['markers'][t], edgecolor=pargs['colors'][t], facecolor=pargs['colors'][t], linewidths=pargs['linewidth'], label=pargs['legends'][t][b], alpha=1)
            # axs[idxes].legend(loc='upper right', fontsize=pargs['fontsizes'][1])
            axs[idxes].set_xlim(-x_max, x_max)
            axs[idxes].set_ylim(0,y_max)
        
            axs[idxes].tick_params(axis='x', which='both', direction='out', length=0, width=0,pad=5, labelsize=pargs['fontsizes'][2], labelbottom=True, labeltop=False, grid_color='k', grid_alpha=1, grid_linewidth=1, grid_linestyle='--')
            axs[idxes].grid(visible=True, which='minor', axis='x', linestyle='--', linewidth=pargs['linewidth'])
            axs[idxes].tick_params(axis='y', which='major', direction='out', length=12, width=4, pad=3, labelsize=pargs['fontsizes'][2], left=True, right=True, labelleft=True, labelright=True)
            axs[idxes].tick_params(axis='y', which='minor', direction='out', length=8, width=4, left=True, right=True, labelleft=False, labelright=False)

    fig.colorbar(h2d[3], cax=axes([1, 0, 0.025, 1]), orientation='vertical', label='Count')
    fig.text(0.5, -0.075, pargs['x_label'], ha='center', fontsize=pargs['fontsizes'][0])
    fig.text(-0.05, 0.5, pargs['y_label'], va='center', rotation='vertical', fontsize=pargs['fontsizes'][0])
    savefig('stats_spat_sim_amp_sim_' + pargs['plot_name'] + '.svg', dpi=300, bbox_inches='tight')
    close()

In [6]:
# plot
data = np.load('1-f-noise-spat-sim-amp-sim.npz')
spat_sim_all, amp_sim_all = data['spat_sim_all'], data['amp_sim_all']
n_row = spat_sim_all.shape[0]
n_col = spat_sim_all.shape[1]
pargs = {'plot_name': '1_f_noise', 'figsize': (16, 8), 'n_rows': n_row, 'n_cols': n_col, 'fontsizes': [36, 15, 12], 'x_label': 'Spatial Cosine Similarity', 'y_label': 'Amplitude Cosine Similarity', 'legends': [['rc | white', 'rc | 1/f^0.5', 'rc | 1/f', 'rc | 1/f^2', 'rc | 1/f^5'], ['sine | white', 'sine | 1/f^0.5', 'sine | 1/f', 'sine | 1/f^2', 'sine | 1/f^5'], ['plaid | white', 'plaid | 1/f^0.5', 'plaid | 1/f', 'plaid | 1/f^2', 'plaid | 1/f^5']], 'markers': ['.', '.', '.', '.'], 'colors': ['k', 'k', 'k', 'k'], 'markersize': 1, 'linewidth': 1}

plot_spat_sim_amp_sim(spat_sim_all, amp_sim_all, pargs)